In [ ]:
#environment setuu

import os
import warnings

# Clean output - suppress warnings
warnings.filterwarnings('ignore')

os.makedirs('medical_rag_bot/docs', exist_ok=True)
os.chdir('/content/medical_rag_bot')

print("✅ Environment initialized")
print("Project folder: medical_rag_bot/")
print("Documents folder: medical_rag_bot/docs/")

✅ Environment initialized
Project folder: medical_rag_bot/
Documents folder: medical_rag_bot/docs/


In [ ]:
#Install Required Packages

print("📥 Installing OpenAI packages...")

!pip install -q openai langchain langchain-openai chromadb PyMuPDF langchain-community

print("✅ All packages installed successfully!")
print("🔗 Installed packages:")
print("   • openai - OpenAI API client")
print("   • langchain - LLM framework")
print("   • langchain-openai - OpenAI integration")
print("   • chromadb - Vector database")
print("   • PyMuPDF - PDF processing")
print("   • langchain-community - Additional tools")

📥 Installing OpenAI packages...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Import Libraries

print("📚 Loading libraries...")

# Core OpenAI imports
import openai
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# LangChain components
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

# Utilities
from getpass import getpass

print("✅ All libraries imported successfully!")

📚 Loading libraries...
✅ All libraries imported successfully!


In [ ]:
# Configure OpenAI API

OPENAI_API_KEY = getpass("Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

print("✅ OpenAI API configured successfully!")
print("🔒 API key securely stored in environment")

Enter your OpenAI API key: ··········
✅ OpenAI API configured successfully!
🔒 API key securely stored in environment


In [ ]:
# Document Upload

from google.colab import files
import shutil

print("📋 Medical Document Upload")
print("─" * 40)
print("📁 Please upload your medical PDF files")
# print("💡 Supported format: PDF only")
# print("⚠️  Large files may take longer to process")
# print("─" * 40)

# File upload interface
uploaded = files.upload()

# Process uploaded files
uploaded_count = 0
for filename in uploaded.keys():
    if filename.endswith('.pdf'):
        shutil.move(filename, f'docs/{filename}')
        print(f"✅ Successfully uploaded: {filename}")
        uploaded_count += 1
    else:
        print(f"❌ Skipped non-PDF file: {filename}")

print(f"\n📊 Upload Summary: {uploaded_count} PDF file(s) uploaded")

📋 Medical Document Upload
────────────────────────────────────────
📁 Please upload your medical PDF files


Saving Medical_book.pdf to Medical_book.pdf
✅ Successfully uploaded: Medical_book.pdf

📊 Upload Summary: 1 PDF file(s) uploaded


In [ ]:
#  Document Processing & Text Extraction

print("📖 Processing Medical Documents")
print("─" * 40)

documents = []
total_pages = 0

# Process each PDF file
for filename in os.listdir("docs"):
    if filename.endswith(".pdf"):
        print(f"🔄 Processing: {filename}")

        # Load PDF and extract text
        loader = PyMuPDFLoader(f"docs/{filename}")
        docs = loader.load()
        documents.extend(docs)

        pages = len(docs)
        total_pages += pages
        print(f"   📄 Extracted {pages} pages")

print("─" * 40)
print(f"📊 Processing Summary:")
print(f"   📚 Total files processed: {len([f for f in os.listdir('docs') if f.endswith('.pdf')])}")
print(f"   📄 Total pages extracted: {total_pages}")


📖 Processing Medical Documents
────────────────────────────────────────
🔄 Processing: Medical_book.pdf
   📄 Extracted 637 pages
────────────────────────────────────────
📊 Processing Summary:
   📚 Total files processed: 1
   📄 Total pages extracted: 637


In [ ]:
# splitting docs into chunks
if documents:
    # Split documents into chunks for better processing
    print(f"\n🔪 Splitting documents into chunks...")
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1200,    # Each chunk ~1200 characters
        chunk_overlap=200   # 200 character overlap for context
    )
    chunks = splitter.split_documents(documents)
    print(f"✅ Created {len(chunks)} text chunks")
    print(f"💡 Chunk size: ~1200 characters each")
else:
    print("❌ No documents found! Please upload PDF files first.")
    chunks = []


🔪 Splitting documents into chunks...
✅ Created 2784 text chunks
💡 Chunk size: ~1200 characters each


In [ ]:
# Vector Database Creation

print("🧬 Creating Vector Database with OpenAI")
print("─" * 40)
print("🔄 Converting text to embeddings...")

# Create OpenAI embeddings model
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # Cost-effective embedding model
    show_progress_bar=True           # Show progress during embedding
)

# Create vector database
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="medical_db"
)

# Create retriever for similarity search
retriever = vectordb.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

print("✅ Vector database created successfully!")
print("💾 Database saved to: medical_db/")


🧬 Creating Vector Database with OpenAI
────────────────────────────────────────
🔄 Converting text to embeddings...


  0%|          | 0/3 [00:00<?, ?it/s]

✅ Vector database created successfully!
💾 Database saved to: medical_db/


In [ ]:
# Language Model Setup

print("🤖 Initializing OpenAI Language Model")
print("─" * 40)

# Initialize ChatGPT model
llm = ChatOpenAI(
    model="gpt-4o-mini",        # Cost-effective, high-performance model
    temperature=0.2,            # Low temperature for consistent, focused responses
    max_tokens=1000,            # Reasonable response length
    timeout=30                  # 30 second timeout
)

print("✅ Language model initialized successfully!")
print("🧠 Model: GPT-4o-mini (optimized for medical queries)")
# print("🌡️  Temperature: 0.2 (focused responses)")
# print("💬 Max tokens: 1000 per response")

🤖 Initializing OpenAI Language Model
────────────────────────────────────────
✅ Language model initialized successfully!
🧠 Model: GPT-4o-mini (optimized for medical queries)


In [ ]:
# Caching System Setup

print("💾 Setting up Response Caching System")
print("─" * 40)

# Simple in-memory cache to avoid repeated API calls
cache = {}

def cached_hybrid_query(query: str):
    """
    Cache responses to avoid repeated API calls and save costs

    Args:
        query (str): User's medical question

    Returns:
        str: Cached or newly generated response
    """
    cache_key = query.lower().strip()

    if cache_key in cache:
        print("Using cached response! (No API cost)")
        return cache[cache_key]

    print("🔄 Processing new query... ")
    result = hybrid_query(query)

    cache[cache_key] = result
    return result

def clear_cache():
    """Clear all cached responses"""
    global cache
    cache = {}
    print("🗑️ Cache cleared! All future questions will use fresh API calls.")

print("✅ Caching system ready!")


💾 Setting up Response Caching System
────────────────────────────────────────
✅ Caching system ready!


In [ ]:
# Retrieval-Augmented Generation (RAG) Chain

print("🔗 Creating Retrieval-Augmented Generation Chain")
print("─" * 40)

# Create RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,                        # Our ChatGPT model
    chain_type="stuff",             # Combine all retrieved docs in one prompt
    retriever=retriever,            # Our vector database retriever
    return_source_documents=True,   # Return sources for citations
    verbose=False                   # Clean output
)

print("✅ RAG chain created successfully!")
print("🔍 Chain type: 'stuff' (combines all relevant documents)")


🔗 Creating Retrieval-Augmented Generation Chain
────────────────────────────────────────
✅ RAG chain created successfully!
🔍 Chain type: 'stuff' (combines all relevant documents)


In [ ]:
# Query Processing Functions

def ask_llm_first(query: str):
    """Try OpenAI directly. Return None if uncertain."""
    system_prompt = f"""
    You are a medical AI assistant. Answer only if you are confident.
    If you don't know, reply exactly: "I don't know."

    Question: {query}
    """

    response = llm.invoke(system_prompt)
    content = response.content if hasattr(response, "content") else str(response)

    if "i don't know" in content.lower():
        return None
    return content


def ask_with_rag(query: str):
    """
    Ask a question using RAG and return the answer with clean source citations.
    Page numbers are corrected for human readability.
    """
    # Get answer from QA chain
    result = qa_chain.invoke({"query": query})
    answer = result.get("result", "No answer found.")

    sources = []
    for doc in result.get("source_documents", []):
        file = os.path.basename(doc.metadata.get("source", "Medical_book.pdf"))

        # Get page number and shift to human-readable
        page = doc.metadata.get("page", "?")
        try:
            page = int(page) + 1
        except:
            page = "?"
        sources.append(f"{file}, p.{page}")
    sources = list(dict.fromkeys(sources))

    return answer, sources


def hybrid_query(query: str):
    """Main function: Try LLM first → fallback to RAG."""
    llm_answer = ask_llm_first(query)
    if llm_answer:
        return f"🧠 **OPENAI**: {llm_answer}"
    else:
        rag_answer, sources = ask_with_rag(query)
        source_list = ", ".join([os.path.basename(s) for s in sources])
        return f"📚 **RAG**: {rag_answer}\n\n📎 **Sources**: {source_list}"

print("✅ Query functions ready!")

✅ Query functions ready!


In [ ]:
# Interactive Command-Line Chat

def start_chat():
    """Interactive command-line medical chatbot"""
    print("🏥 Medical Chatbot with OpenAI")
    print("═" * 40)
    print("💡 Commands: 'exit' to quit, 'cost' for usage, 'cache' for history")
    print("")

    while True:
        user_q = input("💬 **You**: ").strip()

        if user_q.lower() == "exit":
            print("👋 Goodbye!")
            break
        elif user_q.lower() == "cost":
            print(f"💰 Questions answered: {len(cache)}, Estimated cost: ${len(cache) * 0.005:.3f}")
            continue
        elif user_q.lower() == "cache":
            print(f"💾 Cached questions: {len(cache)}")
            continue
        elif not user_q:
            continue

        try:
            answer = cached_hybrid_query(user_q)
            print(f"\n\n{answer}\n")
        except Exception as e:
            print(f"❌ Error: {e}")

print("✅ Command-line chat ready!")

✅ Command-line chat ready!


In [18]:
# Start Interactive Chat

start_chat()

🏥 Medical Chatbot with OpenAI
════════════════════════════════════════
💡 Commands: 'exit' to quit, 'cost' for usage, 'cache' for history

💬 **You**: What percentage of women experience complications that require hospitalization?
🔄 Processing new query... 


  0%|          | 0/1 [00:00<?, ?it/s]



📚 **RAG**: Less than 0.5% of women experience complications from abortions that require hospitalization.

📎 **Sources**: Medical_book.pdf, p.26, Medical_book.pdf, p.354

💬 **You**: In apgar testing, what is the score in most infants
🔄 Processing new query... 


🧠 **OPENAI**: In Apgar testing, most infants typically score between 7 and 10. Scores in this range indicate that the infant is in good health.

💬 **You**: when the  CD4+ lymphocyte count falls below what number, he or  she is at risk for a variety of opportunistic infections?
🔄 Processing new query... 


🧠 **OPENAI**: When the CD4+ lymphocyte count falls below 200 cells/mm³, an individual is at increased risk for a variety of opportunistic infections.

💬 **You**: what is the percentage that PGL affects patients during latency
🔄 Processing new query... 


  0%|          | 0/1 [00:00<?, ?it/s]



📚 **RAG**: PGL affects between 50-70% of patients during latency.

📎 **Sources**: Medical_book.pdf, p.90, Medical_book.pdf, p.98

💬 **You**: What percentage of women experience complications that require hospitalization?
Using cached response! (No API cost)


📚 **RAG**: Less than 0.5% of women experience complications from abortions that require hospitalization.

📎 **Sources**: Medical_book.pdf, p.26, Medical_book.pdf, p.354

💬 **You**: exit
👋 Goodbye!


In [17]:
clear_cache()


🗑️ Cache cleared! All future questions will use fresh API calls.


In [20]:
#  Gradio Web Interface (Optional)

import gradio as gr

def chat_ui(user_input, history=[]):
    """Gradio chat interface function"""
    if not user_input.strip():
        return history, history

    answer = cached_hybrid_query(user_input)
    history = history + [(user_input, answer)]
    return history, history

def show_stats():
    """Show usage statistics"""
    return f"📊 Questions: {len(cache)} | Cost: ${len(cache) * 0.005:.3f}"

# Create Gradio interface
with gr.Blocks(title="Medical RAG Chatbot") as demo:
    gr.Markdown("# 🏥 Medical Chatbot with OpenAI")

    chatbot = gr.Chatbot(label="Medical Assistant", height=400)

    with gr.Row():
        user_input = gr.Textbox(placeholder="Ask your medical question...", scale=4)
        submit_btn = gr.Button("Send", variant="primary", scale=1)

    with gr.Row():
        clear_btn = gr.Button("Clear")
        stats_btn = gr.Button("Stats")
        usage_display = gr.Textbox(label="Usage", interactive=False)

    # Event handlers
    submit_btn.click(chat_ui, [user_input, chatbot], [chatbot, chatbot]).then(lambda: "", outputs=[user_input])
    user_input.submit(chat_ui, [user_input, chatbot], [chatbot, chatbot]).then(lambda: "", outputs=[user_input])
    clear_btn.click(lambda: [], outputs=[chatbot])
    stats_btn.click(show_stats, outputs=[usage_display])

print("✅ Gradio interface ready!")

✅ Gradio interface ready!


In [21]:
# Launch Web Interface

demo.launch(
    share=True,          # Create public link
    show_error=True,     # Show errors in interface
    show_api=False       # Hide API docs
)

print("🚀 Web interface launched!")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://43137af79f15cb34e1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🚀 Web interface launched!
